<a href="https://colab.research.google.com/github/ohdayoung0/llm-fsl-schedule-extractor/blob/main/01_data-collection/Crawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. 크롤링 및 데이터 처리에 필요한 라이브러리 설치
# google-api-python-client: Google Calendar API를 사용하기 위한 공식 라이브러리
# httplib2, oauth2client: API 인증에 필요한 라이브러리 (Colab 기본 모듈 포함)
!pip install requests beautifulsoup4 pandas google-api-python-client oauth2client httplib2

# 2. 라이브러리 임포트
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from datetime import datetime, timedelta
import time # 요청 딜레이를 위한 time 모듈

# Google API 관련 임포트
from google.colab import auth
from googleapiclient.discovery import build
from google.oauth2 import service_account
# 참고: 개인 계정 사용 시 service_account는 필요 없을 수 있으나, 일단 임포트

In [ ]:
import google.auth
from google.colab import auth
from googleapiclient.discovery import build

# 캘린더 이벤트 생성/수정 권한을 요청하는 Scope 정의 (쓰기 권한)
SCOPES = ['https://www.googleapis.com/auth/calendar']

print("Google 계정 인증을 시작합니다...")
# 1. 사용자 로그인 인증 (팝업창이 뜹니다.)
auth.authenticate_user()

# 2. 인증된 사용자로부터 지정된 Scope에 대한 자격 증명을 얻습니다.
credentials, project = google.auth.default(scopes=SCOPES)
print("자격 증명 획득 완료!")

# 3. Calendar API 서비스 객체 생성 시, 획득한 자격 증명(credentials)을 전달합니다.
service = build('calendar', 'v3', credentials=credentials)
print("Calendar API 서비스 객체가 생성되었습니다.")


Google 계정 인증을 시작합니다...
자격 증명 획득 완료!
Calendar API 서비스 객체가 생성되었습니다.


In [ ]:
# 웹사이트 및 크롤링 대상 정의
LIST_URL = 'https://software.cbnu.ac.kr/index.php?mid=sub0401&category=8410'
# 개별 게시물 행의 CSS 선택자 (tr.notice)
NOTICE_SELECTOR = 'tr.notice'
# 목록을 저장할 빈 DataFrame 초기화 (코드 실행 후 데이터가 채워집니다)
df_notices = pd.DataFrame()

# 캘린더 ID 설정
# 이메일 주소 형태의 '기본 캘린더' ID를 사용하거나, 새 캘린더를 생성하여 사용할 수 있습니다.
# 'primary'를 사용하면 로그인한 Google 계정의 기본 캘린더에 일정이 생성됩니다.
CALENDAR_ID = 'primary'

In [ ]:
# 실행 및 결과 확인

# crawl_notice_list 함수 정의
def crawl_notice_list(url):
    try:
        response = requests.get(url)
        response.raise_for_status() # HTTP 오류 발생 시 예외 발생
    except requests.exceptions.RequestException as e:
        print(f"Error during request to {url}: {e}")
        return pd.DataFrame(columns=['Title', 'Link', 'Date']) # 빈 DataFrame 반환

    soup = BeautifulSoup(response.text, 'html.parser')

    notices_data = []
    table = soup.find('table', class_='bd_lst') # 공지사항 목록 테이블 찾기

    if not table:
        print("공지사항 테이블을 찾을 수 없습니다. 선택자를 확인하세요.")
        return pd.DataFrame(columns=['Title', 'Link', 'Date'])

    # 테이블 본문 (tbody) 또는 전체 테이블에서 행(tr) 찾기
    rows = table.find('tbody').find_all('tr') if table.find('tbody') else table.find_all('tr')

    for row in rows:
        cols = row.find_all('td')
        # 최소한 제목과 날짜를 포함하는 컬럼이 있어야 함을 가정
        if len(cols) < 5:
            continue

        title_tag = cols[2].find('a') # 제목과 링크는 보통 3번째 td (인덱스 2)에 위치
        title = ""
        link = ""
        if title_tag:
            title = title_tag.get_text(strip=True)
            link = title_tag.get('href')
            # 상대 경로 링크일 경우 절대 경로로 변환
            if link and not link.startswith('http'):
                base_url_match = re.match(r"(https?://[^/]+)", url)
                if base_url_match:
                    base_url = base_url_match.group(0)
                    link = base_url + link
        else:
            title = cols[2].get_text(strip=True) if len(cols) > 2 else "제목 없음"
            link = "링크 없음"

        date_str = cols[4].get_text(strip=True) if len(cols) > 4 else "날짜 없음"
        date_obj = None
        try:
            # 날짜 형식 'YYYY.MM.DD' 가정
            date_obj = datetime.strptime(date_str, '%Y.%m.%d').date()
        except ValueError:
            pass # 날짜 형식 오류 시 None 유지

        notices_data.append({
            'Title': title,
            'Link': link,
            'Date': date_obj
        })
        time.sleep(0.1) # 요청 간 짧은 지연 추가 (선택 사항, 서버 부담 감소)

    return pd.DataFrame(notices_data)

LIST_URL = 'https://software.cbnu.ac.kr/index.php?mid=sub0401&category=8410'
df_notices = crawl_notice_list(LIST_URL) # <-- 여기서 크롤링이 실행되었습니다!

print("--- 추출된 공지사항 목록 (상위 5개) ---")
print(df_notices.head())

--- 추출된 공지사항 목록 (상위 5개) ---
                                               Title  \
0  글로컬﻿대학30(대학통합) 계속 추진 여부 찬반 투표 공고 (일정 변경 공고 : 투...   
1              2026년도 제 9기 유하푸른재단 장학생 선발 안내 ( ~12.3)   
2    2026학년도 1학기 국가근로장학금 1차 학생 신청 안내 ( ~12.26 18:00)   
3         2026학년도 1학기 국가장학금 1차 신청 안내 ( ~12.26 18:00)   
4      2025년 2학기 연구활동종사자 정기교육 이수 안내 (소프트웨어학부생 전원 해당)   

                                                Link        Date  
0  https://software.cbnu.ac.kr/index.php?mid=sub0...  2025-11-21  
1  https://software.cbnu.ac.kr/index.php?mid=sub0...  2025-11-20  
2  https://software.cbnu.ac.kr/index.php?mid=sub0...  2025-11-17  
3  https://software.cbnu.ac.kr/index.php?mid=sub0...  2025-11-17  
4  https://software.cbnu.ac.kr/index.php?mid=sub0...  2025-11-06  


In [ ]:
print("--- 날짜 정제 결과 (상위 5개) ---")
print(df_notices[['Title', 'Date']].head())

--- 날짜 정제 결과 (상위 5개) ---
                                               Title        Date
0  글로컬﻿대학30(대학통합) 계속 추진 여부 찬반 투표 공고 (일정 변경 공고 : 투...  2025-11-21
1              2026년도 제 9기 유하푸른재단 장학생 선발 안내 ( ~12.3)  2025-11-20
2    2026학년도 1학기 국가근로장학금 1차 학생 신청 안내 ( ~12.26 18:00)  2025-11-17
3         2026학년도 1학기 국가장학금 1차 신청 안내 ( ~12.26 18:00)  2025-11-17
4      2025년 2학기 연구활동종사자 정기교육 이수 안내 (소프트웨어학부생 전원 해당)  2025-11-06


In [ ]:
# 2차 크롤링: 상세 페이지 정보 추출 함수
def crawl_detail_page(url):
    """
    상세 페이지에 접속하여 주최 기관, 장소, 대상을 추출합니다.
    """
    try:
        # 1. 상세 페이지 요청 및 파싱
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        # 2. 공지 본문 컨테이너 찾기 (찾아주신 클래스 사용)
        content_div = soup.select_one('div.xe_content')

        if not content_div:
            return "정보 없음", "정보 없음", "정보 없음"

        # 컨테이너 안의 모든 텍스트를 가져옵니다.
        full_text = content_div.get_text()

        # 3. 정규표현식을 사용하여 정보 추출

        # [주최 기관] 추출 로직: "주최:" 또는 "주관:" 다음에 오는 텍스트
        organizer_match = re.search(r'(?:주최|주관)\s*[:：]\s*(.+?)\s*(?:\n|장소|대상|문의|$)', full_text, re.IGNORECASE | re.DOTALL)
        organizer = organizer_match.group(1).strip() if organizer_match else "정보 없음"

        # [장소/온라인] 추출 로직: "장소:" 또는 "위치:" 다음에 오는 텍스트
        location_match = re.search(r'(?:장소|위치)\s*[:：]\s*(.+?)\s*(?:\n|주최|대상|문의|$)', full_text, re.IGNORECASE | re.DOTALL)
        location = location_match.group(1).strip() if location_match else "온라인 또는 별도 명시 없음"

        # [참여 대상] 추출 로직: "대상:" 다음에 오는 텍스트
        target_match = re.search(r'(?:대상)\s*[:：]\s*(.+?)\s*(?:\n|주최|장소|문의|$)', full_text, re.IGNORECASE | re.DOTALL)
        target = target_match.group(1).strip() if target_match else "전체 학생 또는 별도 명시 없음"

        # 크롤링 부하를 줄이기 위해 짧은 딜레이를 줍니다.
        time.sleep(0.5)

        return organizer, location, target

    except Exception as e:
        print(f"URL {url} 크롤링 중 오류 발생: {e}")
        return "오류", "오류", "오류"

# DataFrame에 새로운 컬럼을 추가하고 함수를 적용합니다.
df_notices[['주최기관', '장소', '대상']] = df_notices['Link'].apply(
    lambda x: pd.Series(crawl_detail_page(x))
)

In [ ]:
print("--- 상세 정보 추출 결과 (상위 5개) ---")
print(df_notices[['Title', '주최기관', '장소', '대상']].head())

--- 상세 정보 추출 결과 (상위 5개) ---
                                               Title   주최기관               장소  \
0  글로컬﻿대학30(대학통합) 계속 추진 여부 찬반 투표 공고 (일정 변경 공고 : 투...  정보 없음  온라인 또는 별도 명시 없음   
1              2026년도 제 9기 유하푸른재단 장학생 선발 안내 ( ~12.3)  정보 없음  온라인 또는 별도 명시 없음   
2    2026학년도 1학기 국가근로장학금 1차 학생 신청 안내 ( ~12.26 18:00)  정보 없음  온라인 또는 별도 명시 없음   
3         2026학년도 1학기 국가장학금 1차 신청 안내 ( ~12.26 18:00)  정보 없음  온라인 또는 별도 명시 없음   
4      2025년 2학기 연구활동종사자 정기교육 이수 안내 (소프트웨어학부생 전원 해당)  정보 없음  온라인 또는 별도 명시 없음   

                                                  대상  
0                                  전체 학생 또는 별도 명시 없음  
1                                  전체 학생 또는 별도 명시 없음  
2                                  전체 학생 또는 별도 명시 없음  
3  재학생, 신입생(고3, 재수생 등 입학 예정자), 편입생, 재입학생, 복학생 등 모...  
4                                  전체 학생 또는 별도 명시 없음  


In [ ]:
from datetime import datetime, timedelta
import pandas as pd # (Colab에 이미 임포트되어 있음)
import re

def extract_deadline_from_title(title):
    """
    제목 문자열에서 마감일 패턴을 추출합니다.
    예: ~12.26 18:00, ~12.3, ~11/24 월
    """
    # '~'로 시작하며 월.일 또는 월/일, 선택적으로 시간(HH:MM), 선택적으로 추가 문자열(요일 등) 포함
    match = re.search(r'~ *(\d{1,2}[./]\d{1,2}(?:\s+\d{1,2}:\d{2})?(?:\s+\S+)?)', title)
    if match:
        return '~' + match.group(1).strip() # 추출된 부분 앞에 '~'를 다시 붙여줍니다.
    return None

def format_deadline(date_str):
    """
    '~12.26 18:00' 형태의 문자열을 ISO 8601 형식으로 변환합니다.
    """
    if date_str is None or '정보 없음' in date_str or not date_str.strip():
        return None

    clean_date_str = date_str.replace('~', '').strip()
    now = datetime.now()
    current_year = now.year

    # 1. 시간 정보가 포함된 날짜 형식 시도 (예: 12.26 18:00, 12/26 18:00)
    try:
        # 시간 정보가 있는지 확인
        if re.search(r'\d{1,2}:\d{2}', clean_date_str):
            temp_date_str = clean_date_str.replace('/', '.') # '/'를 '.'으로 통일
            month_day_time_parts = temp_date_str.split(' ')
            month_day_str = month_day_time_parts[0]
            time_str = month_day_time_parts[1]

            deadline_month = int(month_day_str.split('.')[0])
            target_year = current_year + 1 if deadline_month < now.month else current_year
            full_date_str = f"{target_year}.{month_day_str} {time_str}"
            dt_object = datetime.strptime(full_date_str, '%Y.%m.%d %H:%M')
            return dt_object.isoformat()
    except ValueError:
        pass # 다음 형식 시도

    # 2. 시간 정보가 없는 날짜 형식 시도 (예: 12.3, 11/24 월)
    try:
        # '11/24 월'처럼 요일 정보가 있을 수 있으므로 첫 번째 부분만 사용
        temp_date_str = clean_date_str.split(' ')[0].replace('/', '.')
        month_day_str = temp_date_str

        deadline_month = int(month_day_str.split('.')[0])
        target_year = current_year + 1 if deadline_month < now.month else current_year
        full_date_str = f"{target_year}.{month_day_str}"
        dt_object = datetime.strptime(full_date_str, '%Y.%m.%d')
        # 시간 정보가 없는 경우, 마감일을 해당 날짜의 끝으로 설정
        return (dt_object + timedelta(days=1) - timedelta(seconds=1)).isoformat()
    except ValueError:
        pass

    # 모든 시도가 실패한 경우
    print(f"날짜 변환 오류 발생 (지원하지 않는 패턴): {date_str}")
    return None

# DataFrame에 '마감일_원문' 컬럼 추가 (Title에서 마감일 문자열 추출)
df_notices['마감일_원문'] = df_notices['Title'].apply(extract_deadline_from_title)

# DataFrame에 '마감일_정제' 컬럼 추가 (원문 마감일 정제)
df_notices['마감일_정제'] = df_notices['마감일_원문'].apply(format_deadline)


In [ ]:
print("--- 날짜 정제 결과 (상위 5개) ---")
# '장소', '주최기관'은 정보 없음이 나왔으므로, 제목과 날짜만 확인합니다.
print(df_notices[['Title', '마감일_원문', '마감일_정제']].head())

--- 날짜 정제 결과 (상위 5개) ---
                                               Title        마감일_원문  \
0  글로컬﻿대학30(대학통합) 계속 추진 여부 찬반 투표 공고 (일정 변경 공고 : 투...          None   
1              2026년도 제 9기 유하푸른재단 장학생 선발 안내 ( ~12.3)         ~12.3   
2    2026학년도 1학기 국가근로장학금 1차 학생 신청 안내 ( ~12.26 18:00)  ~12.26 18:00   
3         2026학년도 1학기 국가장학금 1차 신청 안내 ( ~12.26 18:00)  ~12.26 18:00   
4      2025년 2학기 연구활동종사자 정기교육 이수 안내 (소프트웨어학부생 전원 해당)          None   

                마감일_정제  
0                 None  
1  2025-12-03T23:59:59  
2  2025-12-26T18:00:00  
3  2025-12-26T18:00:00  
4                 None  


In [ ]:
# 현재 사용자의 기본 캘린더 ID (이전에 설정함)
CALENDAR_ID = 'primary'
# 한국 시간대 설정 (Colab 환경에서 KST를 사용한다고 가정)
TIMEZONE = 'Asia/Seoul'

def create_calendar_event(row, service, calendar_id):
    """
    DataFrame의 한 행(Row) 데이터를 Google Calendar 이벤트로 생성합니다.
    """
    # 마감일이 정제되지 않았으면(None) 이벤트 생성 스킵
    if pd.isna(row['마감일_정제']):
        return f"SKIP: {row['Title']} (날짜 정보 없음)"

    # 주최기관/장소 정보가 없으므로 상세 정보를 Description에 모두 포함
    description = (
        f" 대상: {row['대상']}\n"
        f" 주최: {row['주최기관']}\n"
        f" 장소: {row['장소']}\n"
        f"🔗 원본 링크: {row['Link']}"
    )

    # 1. 캘린더 이벤트 본문 생성
    event = {
        'summary': f"[DEADLINE] {row['Title']}",  # 캘린더 제목
        'location': row['장소'],                  # 장소 (없으면 '온라인 또는 별도 명시 없음' 사용)
        'description': description,              # 상세 설명
        'start': {
            'dateTime': row['마감일_정제'],
            'timeZone': TIMEZONE,
        },
        'end': {
            'dateTime': row['마감일_정제'],       # 마감일 이벤트는 시작/종료 시간을 동일하게 설정
            'timeZone': TIMEZONE,
        },
        'reminders': {
            'useDefault': False,
            'overrides': [
                {'method': 'popup', 'minutes': 60 * 24}, # 하루 전 알림
                {'method': 'popup', 'minutes': 60 * 2},  # 2시간 전 알림
            ],
        },
    }

    try:
        # 2. API 호출하여 이벤트 삽입
        event = service.events().insert(calendarId=calendar_id, body=event).execute()
        return f"SUCCESS: {row['Title']} (ID: {event.get('id')})"

    except Exception as e:
        return f"FAILED: {row['Title']} (Error: {e})"


# DataFrame을 순회하며 캘린더 이벤트 생성
print("--- Google Calendar 이벤트 생성 시작 ---")
results = []
# df_notices는 1차 크롤링 결과를 담고 있는 DataFrame이어야 합니다.
for index, row in df_notices.iterrows():
    result = create_calendar_event(row, service, CALENDAR_ID)
    results.append(result)
    print(result) # 실시간 진행 상황 출력
    time.sleep(1) # API 요청 과부하 방지를 위해 1초 딜레이

print("--- Google Calendar 이벤트 생성 완료 ---")

--- Google Calendar 이벤트 생성 시작 ---
SKIP: 글로컬﻿대학30(대학통합) 계속 추진 여부 찬반 투표 공고 (일정 변경 공고 : 투표일정 연기) (날짜 정보 없음)


FAILED: 2026년도 제 9기 유하푸른재단 장학생 선발 안내 ( ~12.3) (Error: <HttpError 403 when requesting https://www.googleapis.com/calendar/v3/calendars/primary/events?alt=json returned "Request had insufficient authentication scopes.". Details: "[{'message': 'Insufficient Permission', 'domain': 'global', 'reason': 'insufficientPermissions'}]">)


FAILED: 2026학년도 1학기 국가근로장학금 1차 학생 신청 안내 ( ~12.26 18:00) (Error: <HttpError 403 when requesting https://www.googleapis.com/calendar/v3/calendars/primary/events?alt=json returned "Request had insufficient authentication scopes.". Details: "[{'message': 'Insufficient Permission', 'domain': 'global', 'reason': 'insufficientPermissions'}]">)


FAILED: 2026학년도 1학기 국가장학금 1차 신청 안내 ( ~12.26 18:00) (Error: <HttpError 403 when requesting https://www.googleapis.com/calendar/v3/calendars/primary/events?alt=json returned "Request had insufficient authentication scopes.". Details: "[{'message': 'Insufficient Permission', 'domain': 'global', 'reason': 'insufficientPermissions'}]">)
SKIP: 2025년 2학기 연구활동종사자 정기교육 이수 안내 (소프트웨어학부생 전원 해당) (날짜 정보 없음)
SKIP: 2025학년도 한국열린사이버대학교(OCU) 동기 계절수업 안내 (날짜 정보 없음)


FAILED: 2025학년도 2학기 산학프로젝트, 창업파일럿프로젝트(종합설계) 재료비 신청 안내(~12/31) (Error: <HttpError 403 when requesting https://www.googleapis.com/calendar/v3/calendars/primary/events?alt=json returned "Request had insufficient authentication scopes.". Details: "[{'message': 'Insufficient Permission', 'domain': 'global', 'reason': 'insufficientPermissions'}]">)
SKIP: 2025학년도 동기 계절수업 운영 계획 안내 (날짜 정보 없음)
SKIP: [빅데이터 융합전공/중요] 2026년도 2월 졸업예정자 졸업논문 제출 안내 (날짜 정보 없음)
SKIP: 「교양영어 선택이수제」 안내 (날짜 정보 없음)
SKIP: 2025-2학기 산학프로젝트 교과목 수강신청을 위한 안내 사항 (날짜 정보 없음)
SKIP: [중요] 소프트웨어학부 개신프론티어 교과 선발을 위한 학부 심의 기준 안내 (날짜 정보 없음)
SKIP: 소프트웨어학부 GPU 서버 대여 안내 (2025년 6월 10일부터 시행) (날짜 정보 없음)
SKIP: 학생 집단활동(MT, 동아리 활동 등) 사전 허가 필수 안내(학생 안전사고 유의) (날짜 정보 없음)
SKIP: 출석(공결)인정 절차 변경 안내(6/12~) (날짜 정보 없음)
SKIP: 공결관련 참고사항 (날짜 정보 없음)
SKIP: 신입생(학부 및 대학원생) 모바일 학생증 사용방법 안내 (날짜 정보 없음)
SKIP: 전과생 및 편입생의 수강신청 방법 변경 안내(추가) (날짜 정보 없음)
SKIP: 알고리즘(2-2), 객체지향설계(3-1) 교과 선수과목 변동 안내 (날짜 정보 없음)
SKIP: [필독] 소프트웨어학부 학부장 승인관련 안내 (날짜 정보 없음)
SKIP: 2022-2024 다부전공 신청 학생 교육과

...

FAILED: 2025년 충북지역 대학생 인공지능 활용 아이디어 공모전 모집 공고(~11/9 일) (Error: <HttpError 403 when requesting https://www.googleapis.com/calendar/v3/calendars/primary/events?alt=json returned "Request had insufficient authentication scopes.". Details: "[{'message': 'Insufficient Permission', 'domain': 'global', 'reason': 'insufficientPermissions'}]">)
SKIP: 2025년 하반기 '전국 에너지 공동학점과정' 안내 (날짜 정보 없음)
SKIP: 졸업(예정자) 대상 소프트웨어 기술자 신청 수수료 감면 안내 (날짜 정보 없음)
SKIP: [KISIA] 2025 정보보호 정책제안 공모전 (날짜 정보 없음)


FAILED: 세계 최초 인공지능 신뢰성 해거톤 대회 TRAITHON 2025 개최 안내(접수: ~9/26 금 까지) (Error: <HttpError 403 when requesting https://www.googleapis.com/calendar/v3/calendars/primary/events?alt=json returned "Request had insufficient authentication scopes.". Details: "[{'message': 'Insufficient Permission', 'domain': 'global', 'reason': 'insufficientPermissions'}]">)


FAILED: 2026학년도 1학기 영어권 SAF 자매대학 파견 교환학생 선발 안내 ( ~9.12) (Error: <HttpError 403 when requesting https://www.googleapis.com/calendar/v3/calendars/primary/events?alt=json returned "Request had insufficient authentication scopes.". Details: "[{'message': 'Insufficient Permission', 'domain': 'global', 'reason': 'insufficientPermissions'}]">)
SKIP: 2025학년도 2학기 표준현장실습 학생 매칭 안내-아이티엠반도체 (날짜 정보 없음)


FAILED: 2025학년도 2학기 외국인 교환학생 버디앤유(Buddy&U) 프로그램 ( ~8.31) (Error: <HttpError 403 when requesting https://www.googleapis.com/calendar/v3/calendars/primary/events?alt=json returned "Request had insufficient authentication scopes.". Details: "[{'message': 'Insufficient Permission', 'domain': 'global', 'reason': 'insufficientPermissions'}]">)
SKIP: 2025년 Global Innovators’ IT LAB 해커톤 대회 모집 공고 (날짜 정보 없음)
SKIP: [중부정보보호지원센터] 2025 정보보호 무료 온라인 교육 안내 (날짜 정보 없음)


FAILED: SK쉴더스 채용연계형 교육생 모집 ( ~8.24) (Error: <HttpError 403 when requesting https://www.googleapis.com/calendar/v3/calendars/primary/events?alt=json returned "Request had insufficient authentication scopes.". Details: "[{'message': 'Insufficient Permission', 'domain': 'global', 'reason': 'insufficientPermissions'}]">)
--- Google Calendar 이벤트 생성 완료 ---
